In [1]:
import os
import time
from model import *
from data import *

Using TensorFlow backend.


In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

![U-NET](https://lmb.informatik.uni-freiburg.de/Publications/2015/RFB15a/u-net-architecture.png)


U-Net，基於Fully Convolutional Networks(全捲積網路)，輸入和輸出都是圖，沒有全連接層。
https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/

## Train your Unet with membrane data
membrane data is in folder Membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [3]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [15]:
tic = time.time()

BatchSize_per_Replica = 1
BatchSize = BatchSize_per_Replica * strategy.num_replicas_in_sync
print(BatchSize)
#BatchSize = 1

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(BatchSize,'Membrane','image','label',data_gen_args,save_to_dir = None)
#myGene = trainGenerator(BatchSize,'Fixed','image','label',data_gen_args,save_to_dir = None)
#myGene = trainGenerator(BatchSize,'Mitochondrion','image','label',data_gen_args,save_to_dir = None)

with strategy.scope():
    model = unet()
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint('UNet_Membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    #model_checkpoint = ModelCheckpoint('UNet_Fixed.hdf5', monitor='loss',verbose=1, save_best_only=True)
    #model_checkpoint = ModelCheckpoint('UNet_Mitochondrion.hdf5', monitor='loss',verbose=1, save_best_only=True)
    #model.fit_generator(myGene,steps_per_epoch=2000,epochs=10,callbacks=[model_checkpoint])
    model.fit(myGene,steps_per_epoch=1000,epochs=5,callbacks=[model_checkpoint,tf.keras.callbacks.TensorBoard(log_dir='./logs')])

toc = time.time()
print("Time=" + str((toc-tic)) + "sec")

2
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 48 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 48 all-reduces with algorithm = nccl, num_packs = 1
   1/1000 [..............................] - ETA: 0s - loss: 0.6824 - accuracy: 0.3795WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1000/1000 [==============================] - ETA: 0s - loss: 0.2694 - accuracy: 0.9020
Epoch 00001: loss improved from inf to 0.26939, saving model to UNet_Membrane.hdf5
1000/1000 [==============================] - 171s 171ms/step - loss: 0.2694 - accuracy: 0.9020
Epoch 2/5
1000/1000 [====

### test your model and save predicted results

In [4]:
testGene = testGenerator("Membrane/test")
model = unet()
model.load_weights("UNet_Membrane.hdf5")
#results = model.predict_generator(testGene,30,verbose=1)
results = model.predict(testGene,30,verbose=1)
saveResult("Membrane/test",results)

      1/Unknown - 0s 35us/stepWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0034s vs `on_predict_batch_end` time: 0.0436s). Check your callbacks.
30/30 [==============================] - 1s 43ms/step


### Train with npy file

In [11]:
BatchSize_per_Replica = 1
BatchSize = BatchSize_per_Replica * strategy.num_replicas_in_sync
print(BatchSize)

2


In [5]:
imgs_train,imgs_mask_train = geneTrainNpy("Membrane/aug/","Membrane/aug/")

In [7]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('UNet_Membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)

In [14]:
with strategy.scope():
    model_2gpu = unet()
    model_2gpu.fit(imgs_train, imgs_mask_train, batch_size=BatchSize, steps_per_epoch=1000, epochs=5, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

Epoch 1/5
INFO:tensorflow:batch_all_reduce: 48 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 48 all-reduces with algorithm = nccl, num_packs = 1
 120/1000 [==>...........................] - ETA: 2:24 - loss: 0.3859 - accuracy: 0.8228WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 5000 batches). You may need to use the repeat() function when building your dataset.

Epoch 00001: loss did not improve from 0.28946
 120/1000 [==>...........................] - 22s 181ms/step - loss: 0.3859 - accuracy: 0.8228 - val_loss: 0.3257 - val_accuracy: 0.8658
